In [ ]:
import os
import faiss
from Libraries import A0_MyUtils as A0, A1_TextProcess as A1, A2_PdfProcess as A2
from Libraries import B1_ExtractData as B1, B2_MergeData as B2, B3_GetStructures as B3
from Libraries import B4_ChunkMaster as B4, B5_ChunkFlex as B5, B6_ChunkFixed as B6
from Libraries import C1_CreateSchema as C1, C2_Embedding as C2, C3_CheckConstruct as C3
from Libraries import D0_FaissConvert as D0, D1_Search as D1, D2_Rerank as D2, D3_Respond as D3
from Config import Widgets, Configs

In [ ]:
os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"
widgets_list = Widgets.create_name_form()

In [ ]:
config = Configs.WidgetValues(widgets_list)

data_foler = config["data_folder"]
dcmt_path = config["dcmt_path"]
base_folder = config["base_folder"]
base_path = config["base_path"]
extracted_path = config["extracted_path"]
merged_path = config["merged_path"]
struct_path = config["struct_path"]
chunks_base = config["chunks_base"]
chunks_segment = config["chunks_segment"]
schema_ex_path = config["schema_ex_path"]
embedding_path = config["embedding_path"]
torch_path = config["torch_path"]
faiss_path = config["faiss_path"]
mapping_path = config["mapping_path"]
mapping_data = config["mapping_data"]

FILE_TYPE = config["FILE_TYPE"]
DATA_KEY = config["DATA_KEY"]
EMBE_KEY = config["EMBE_KEY"]
SWITCH = config["SWITCH"]
EMBEDD_MODEL = config["EMBEDD_MODEL"]
SEARCH_EGINE = config["SEARCH_EGINE"]
RERANK_MODEL = config["RERANK_MODEL"]
RESPON_MODEL = config["RESPON_MODEL"]
MERGE = config["MERGE"]
API_KEY = config["API_KEY"]

WORD_LIMIT = config["WORD_LIMIT"]
LEVEL_INPUT = config["LEVEL_INPUT"]
LEVEL_VALUES = config["LEVEL_VALUES"]

Contents = LEVEL_VALUES[-1] if LEVEL_VALUES else None

SEARCH_ENGINE = faiss.IndexFlatIP

### Exceptions

In [ ]:
assets = "../Assets/"
exceptions_path = f"{assets}ex.exceptions.json"
markers_path = f"{assets}ex.markers.json"
status_path = f"{assets}ex.status.json"

### Extract Data

In [ ]:
dataExtractor = B1.B1Extractor(
    exceptions_path, 
    markers_path, 
    status_path, 
    proper_name_min_count=10)

extracted_data = dataExtractor.extract(dcmt_path)
A0.write_json(extracted_data, extracted_path, indent=1)

In [ ]:
merged_data = B2.mergeLinesToParagraphs(extracted_data)
A0.write_json(merged_data, merged_path, indent=1)

### Get Struct

In [ ]:
analyzer = B3.StructureAnalyzer(merged_path, verbose=True)

In [ ]:
def getMarkers():
    markers = analyzer.extract_markers()
    return markers

In [ ]:
def getStructures(markers):
    structures = analyzer.build_structures(markers)
    print(A0.jsonl_convert(structures))
    return structures

In [ ]:
def getDedup(structures):
    dedup = analyzer.deduplicate(structures)
    print(A0.jsonl_convert(dedup))
    return dedup

In [ ]:
def getTop(dedup):
    top = analyzer.select_top(dedup)
    topext = analyzer.extend_top(top, dedup)
    print(A0.json_convert(topext, pretty=True))
    return topext

In [ ]:
markers = getMarkers()
structures = getStructures(markers)
dedup = getDedup(structures)
topext = getTop(dedup)

In [ ]:
A0.write_json(topext, struct_path, indent=2)

### Chunks

In [ ]:
builder = B4.ChunkBuilder(
    struct_path=struct_path,
    merged_path=merged_path,
    output_path="Data_HNMU_Regulations_Chunks.json"
)

chunks = builder.build()
A0.write_json(chunks, chunks_base, indent=2)